In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
#Función que permite "parsear" archivos F24 de OPTA (XML)
def eventos_opta(nombre_archivo):
    import xml.etree.ElementTree as ET
    import pandas as pd
    import os
    import glob
    import numpy as np
    
    global archivo
    
    tree = ET.ElementTree(file = nombre_archivo)
    archivo = tree.getroot()
    
    lista_principal = []

    for partido in archivo:
        id_visitante = partido.attrib.get("away_team_id")
        id_local = partido.attrib.get("home_team_id")
        visitante = partido.attrib.get("away_team_name")
        local = partido.attrib.get("home_team_name")
        fecha = partido.attrib.get('matchday')
        gol_local = partido.attrib.get('home_score')
        gol_visitante = partido.attrib.get('away_score')
        
        for evento in partido:
             for q in evento:
                 qualifier = q.attrib.get("qualifier_id")
                 if qualifier == "277":
                     tiempo = evento.attrib.get("period_id")
                     if tiempo == "1":
                         Stoppage_1 = (q.attrib.get("value"))
                     if tiempo == "2":
                         Stoppage_2 = (q.attrib.get("value"))
                            
                 variables = {"y": evento.attrib.get("y"),
                              "x": evento.attrib.get("x"),
                              "outcome": evento.attrib.get("outcome"),
                              "id_equipo": evento.attrib.get("team_id"),
                              "id_jugador": evento.attrib.get("player_id"),
                              "segundo": evento.attrib.get("sec"),
                              "minuto": evento.attrib.get("min"),
                              "id_periodo": evento.attrib.get("period_id"),
                              "id_categoria_evento": evento.attrib.get("type_id"),
                              "id_evento": evento.attrib.get("event_id"),
                              "id_calificador": q.attrib.get("qualifier_id"),
                              "valor": q.attrib.get("value"),
                              'id_local':id_local,
                              'id_visitante':id_visitante,
                              'local':local,
                              'visitante':visitante,
                              'gol_local':gol_local,
                              'gol_visitante':gol_visitante,
                              'fecha':fecha}
                                  
                 lista_principal.append(variables)     
                
    eventos = pd.DataFrame(lista_principal)
    return (eventos)

In [3]:
#Función que permite obtener lista de jugadores a partir de archivo F9 de OPTA (XML)
def jugadores_opta(archivo2):
    import xml.etree.ElementTree as et
    tree2 = et.ElementTree(file = archivo2)
    archivo2 = tree2.getroot()
    id_jugador = []
    nombre_jugador = []

    for child in archivo2:

        for grchild in child:
            
            if grchild.tag == "Team":
                
                for grgrchild in grchild:
                    
                    if grgrchild.tag == "Player":
                        
                        id_jugador.append(grgrchild.attrib["uID"].lstrip('p'))
                        
                        for grgrgrchild in grgrchild:
                            
                            nombre_jugador.append(grgrgrchild[0].text + " " + grgrgrchild[-1].text)

    jugadores = dict(zip(id_jugador,nombre_jugador))
    return (jugadores)

In [4]:
# Se obtiene la lista de archivos en la carpeta "datos_OPTA" donde se encuentran los archivos F24 y F9.
# Se cuenta con 380 archivos F24 y 380 archivos F9.
archivos = [f for f in listdir(r'C:\Users\DELUX\Desktop\consultoria_futbol\analitica\rawdata\OPTA\laLiga_1819') if isfile(join(r'C:\Users\DELUX\Desktop\consultoria_futbol\analitica\rawdata\OPTA\laLiga_1819', f))]

In [5]:
# Lista de archivos F24.
f24 = archivos[0:380]

In [6]:
cd C:\Users\DELUX\Desktop\consultoria_futbol\analitica\rawdata\OPTA\laLiga_1819

C:\Users\DELUX\Desktop\consultoria_futbol\analitica\rawdata\OPTA\laLiga_1819


In [7]:
# Se aplica la 1era función al conjunto de archivos F24.
eventos = []
for e in f24:
    partido = eventos_opta(e)
    eventos.append(partido)

In [8]:
# Dataframe con la totalidad de datos de LaLiga (temporada 2018-2019)
laLiga = pd.concat(eventos, axis=0)

In [9]:
# Lista de archivos F9.
f9 = archivos[-383:-3]

In [10]:
# Se aplica la 2da función al conjunto de archivos F9.
jug = []
for e in f9:
    jugador = jugadores_opta(e)
    jug.append(jugador)
dicc_jugadores = {}
for d in jug:
    dicc_jugadores.update(d)

In [11]:
# Dataframe.
# id_categoria_evento:categoria_evento
dicc_eventos = {'1':'pase','2':'pase en offside','3':'drible','4':'falta','5':'balón sale','6':'tiro de esquina',
               '7':'tacle','8':'intercepción','9':'error no forzado / pérdida de posesión','10':'salvada del portero',
                '11':'portero descuelga centro','12':'despeje','13':'remate fallido','14':'poste','15':'tiro salvado',
                '16':'gol','17':'tarjeta','18':'jugador sale','19':'jugador entra','20':'jugador retirdo (no hay cambios)',
                '21':'jugador vuelve','22':'jugador se vuelve portero','23':'portero se vuelve jugador',
                '24':'cambio condiciones de juego','25':'cambio de árbitro','27':'reinicio demorado','28':'fin demora',
                '30':'fin de tiempo','32':'inicio de tiempo','34':'alineación de equipo','35':'jugador cambia de posición',
                '36':'jugador cambia camiseta','37':'fin de partido','38':'gol (pendiente)','39':'remate (pendiente)',
                '40':'cambio de formación','41':'despeje con puños','42':'despliegue de habilidad','43':'evento borrado',
                '44':'duelo aéreo','45':'jugador gambeteado','47':'tarjeta anulada','49':'recuperación de balón',
                '50':'jugador tacleado','51':'error provoca remate','52':'portero recoge el balón',
                '53':'portero falla descolgar','54':'portero gana posesión','55':'offside provocado',
                '56':'cubre balón con cuerpo','57':'mal saque lateral','58':'portero en penal',
                '59':'salida despeje de portero','60':'oportunidad desperdiciada','61':'mala recepción',
                '63':'salvada (pendiente)','64':'partido reanudado','65':'decisión contenciosa','68':'árbitro bota el balón',
                '70':'tiempo muerto por lesión','71':'configuración de entrenador','72':'jugador en offside',
                '74':'pase bloqueado','75':'partido comienza demorado','76':'partido termina temprano',
                '77':'jugador fuera de terreno de juego','79':'interrupción de señal'}

In [12]:
# Dataframe.
# id_calificador:calificador
dicc_calificadores = {'1':'balón largo','2':'centro','3':'pase de cabeza','4':'balón en profundidad','5':'tiro libre',
                     '6':'tiro de esquina','7':'jugador en offside','8':'pase gol anulado','107':'saque lateral','140':'x_1',
                     '141':'y_1','152':'tiro libre directo','155':'pase globo','156':'pase en dirección corrida',
                     '157':'pase a la zona','168':'pase acomodado con la cabeza','195':'pase hacía atrás en área',
                     '196':'cambio de frente','210':'asistencia a remate','212':'distancia de balón','213':'angulo de balón',
                     '218':'pase para segunda asistencia','223':'tiro de esquina hacía adentro',
                      '224':'tiro de esquina hacía afuera','225':'tiro de esquina recto','236':'pase bloquedo',
                      '238':'fairplay','240':'comienzo del portero','241':'tiro libre indirecto','279':'saque del centro',
                      '278':'toque de balón','287':'portero pasa con brazo','345':'centro bueno',
                      '346':'siguiente evento es saque de meta','347':'siguiente evento es lateral',
                      '23':'pase offside en contraataque','297':'pase offside remate con rebote',
                     '298':'pase offside remate bloqueado','15':'cabeza','72':'pie izquierdo','20':'pie derecho',
                     '21':'otra parte del cuerpo','22':'remate en jugada regular','23':'remate en contraataque',
                     '24':'remate en jugada a balón parado','25':'remate a partir de tiro esquina','26':'remate de tiro libre',
                     '29':'remate asistido','55':'id evento relevante','154':'asistencia intencional',
                     '160':'remate en jugada de lateral','233':'id evento contrario relevante','9':'penal','28':'auto gol',
                     '108':'volea','113':'remate fuerte','114':'remate débil','120':'remate con desvío izquierda',
                     '121':'remate con desvío derecha','122':'remate con varios desvíos','133':'remate desviado en jugador',
                     '136':'gol tocado por portero','137':'remate salvado por portero','138':'remate al palo',
                     '153':'remate fallido que no sale','214':'gran oportunidad de gol','215':'jugada individual',
                     '217':'remate segunda asistencia','228':'remate bloqueado por propio compañero',
                     '230':'x portero en gol o poste','231':'y portero en gol o poste','254':'gol con gambeta',
                     '263':'gol olímpico','316':'penal con pase','328':'remate de primera','16':'área chica al centro',
                     '17':'área grande al centro','18':'fuera del área al centro','19':'más de 35 al centro',
                     '60':'área chica a la derecha','61':'área chica a la izquierda','62':'área grande al fondo derecha',
                     '63':'área grande a la derecha','64':'área grande a la izquierda','65':'área grande al fondo izquierda',
                     '66':'fuera del área al fondo derecha','67':'fuera del área a la derecha',
                     '68':'fuera del área a la izquierda','69':'fuera del área al fondo izquierda',
                     '70':'más de 35 a la derecha','71':'más de 35 a la izquierda','73':'balón sale por la izquierda',
                     '74':'balón sale por arriba','75':'balón sale por la derecha','76':'gol por abajo a la izquierda',
                     '77':'gol por arriba a la izquierda','78':'gol por abajo medio','79':'gol por arriba medio',
                     '80':'gol por abajo a la derecha','81':'gol por arriba a la derecha','82':'bloquedo',
                     '83':'cerca izquierda','84':'cerca derecha','85':'cerca arriba','86':'cerca arriba izquierda',
                     '87':'cerca arriba derecha','100':'remate bloqueado en yarda 6','101':'remate salvado en línea de gol',
                     '102':'y_gol','103':'z_gol','146':'x remate bloqueado','147':'y remate bloqueado',
                     '276':'remate fallido al lateral','300':'corrida en solitario','10':'falta por mano',
                     '11':'falta portero regla 6 segundos','12':'falta por jugada peligrosa','13':'falta',
                      '31':'tarjeta amarilla','32':'segunda tarjeta amarilla','33':'tarjeta roja',
                      '34':'tarjeta por abusar de árbitro','35':'tarjeta por discutir','36':'tarjeta por conducta violenta',
                      '37':'tarjeta por perder tiempo','38':'tarjeta por celebración excesiva',
                      '39':'tarjeta por interacción con público','40':'tarjeta por otra razón',
                      '95':'falta portero recibe pase con mano','132':'simulación de falta',
                      '158':'tarjeta por reiteración de faltas','159':'tarjeta por lenguaje abusivo',
                     '161':'tarjeta por obstaculizar tiro libre','162':'tarjeta por salir del campo sin permiso',
                     '163':'tarjeta por entrar al campo sin permiso','164':'tarjeta por escupir',
                     '165':'tarjeta por falta técnica','166':'tarjeta por salvar gol con mano',
                     '171':'tarjeta anulada post partido','172':'tarjeta a jugador en banquillo que no entra',
                     '184':'tarjeta por desobedecer al árbitro','191':'tarjeta por falta a jugador sin balón',
                     '192':'falta por bloquear remate con mano','241':'falta indirecta','242':'falta por obstrucción',
                     '243':'tarjeta por comportamiento antideportivo','244':'tarjeta por no retroceder en balón parado',
                     '245':'tarjeta por falta fuerte','264':'falta aérea','265':'falta por intento de tacle',
                     '289':'falta para evitar oportunidad de gol','294':'falta por empujar','295':'falta por jalar camiseta',
                     '296':'falta por usar codo o jugada violenta','313':'falta por reinicio de jugada ilegalmente',
                     '314':'reinicio por remate que golpea a jugador en offside','190':'portero salva remate que salía',
                      '88':'portero descuelga centro','89':'atacante en uno a uno con portero',
                      '123':'portero saca balón por lateral','124':'pase de saque de meta',
                      '128':'portero despeja con puños','139':'portero salva remate desviado en compañero',
                      '173':'portero rechaza remate a lugar seguro','174':'portero rechaza remate a rival',
                      '175':'portero salva con los dedos','176':'portero agarra balón',
                      '177':'portero salva y gana posesión de balón','178':'portero salva quieto','179':'portero salva volando',
                      '180':'portero salva agachado','181':'portero salva estirándose por balón',
                      '182':'portero salva con manos','183':'portero salva con pies',
                      '198':'portero despeja con pie dejando balón en suelo','199':'portero despeja con mano-pie',
                      '237':'saque de meta bajo','273':'choca en poste derecho luego de salvada',
                      '274':'choca en poste izquierdo luego de salvada','275':'choca en larguero luego de salvada',
                      '232':'portero agarra balón sin presión','301':'portero salva remate de centro',
                      '186':'portero recibe gol de penal','187':'portero salva penal','188':'portero no salva, penal desviado',
                      '14':'acción defensiva en última línea','94':'jugador bloquea remate','167':'tacle o despeje saca balón',
                      '169':'error provoca remate a portería','170':'error provoca gol','185':'centro bloqueado',
                      '239':'tiro libre bloqueado por barrera','285':'duelo defensivo','286':'duelo ofensivo','211':'overrun',
                      '234':'posesión local','235':'posesión visitante','288':'tiempo sin jugar',
                      '30':'equipo titular y suplente','41':'sustitución por lesión','42':'sustitución táctica',
                      '44':'posición de jugador','59':'número de la camiseta','130':'formación del equipo',
                      '131':'formación de jugador en el equipo','145':'lugar en la formacion','194':'capitán',
                      '197':'indumentaria del equipo','283':'id_entrenador','290':'roles de entrenadores',
                      '292':'posición detallada','293':'lugar de la posición','319':'cambio de capitán',
                      '50':'posición de árbitros','51':'id árbitros','200':'árbitro para jugada','201':'árbitro demora jugada',
                      '208':'árbitro lesionado','247':'árbitro toma decisión contenciosa por offside',
                      '248':'árbitro toma decisión contenciosa en jugada de gol','53':'id jugador lesionado',
                      '202':'clima interrumpe partido','203':'público interrumpe partido','204':'fuego interrumpe partido',
                      '205':'objetos arrojados interrumpen partido','206':'público invade cancha interrumpe partido',
                      '207':'árbitro demora decisión','226':'partido se suspende',
                      '227':'partido se reanuda luego de suspensión','246':'pausa por hidratación','299':'demora afecta reloj',
                      '303':'falla en los reflectores','344':'pérdido de señal','54':'causa de la finalización de partido',
                      '189':'evento no visible','209':'final del partido','277':'minutos que se adicionan',
                      '46':'condiciones de juego','47':'estado de la cancha','48':'luces',
                      '49':'cantidad de gente en el estadio','255':'apertura del techo del estadio','256':'humedad en el aire',
                      '257':'presión del aire','258':'se agotaron las entradas','259':'temperatura en grados centigrados',
                      '260':'iluminación artificial','304':'balón en juego con jugador fuera de la cancha',
                      '305':'juego no reiniciado con jugador fuera de la cancha',
                      '306':'jugador fuera de la cancha por cambio de indumentaria','310':'jugador sale de la cancha',
                      '311':'jugador vuelve a la cancha','348':'jugador que patea penal','127':'dirección del juego',
                      '56':'zona','144':'explicación de eventos borrados','250':'bloqueo (pendiente)',
                      '57':'tipo de finalización','193':'medida de gol','343':'revisión de la jugada',
                      '336':'decisión cancelada','329':'Gol VAR','330':'VAR penal concedido','331':'VAR penal no concedido',
                      '332':'VAR tarjeta más dura','333':'identidad confundida','334':'VAR otro asunto',
                      '335':'decisión de árbitro confirmada','341':'Gol no condedido','342':'tarjeta roja',
                      '281':'asistencia de fantasía','282':'equipo asistencia de fantasia',
                      '280':'tipo de asistencia de fantasia','216':'2do id evento relacionado','252':'tiro errado (pendiente)',
                      '253':'tiro errado sale última línea (pendiente)','251':'tiro al poste (pendiente)',
                      '307':'fase de posesion','249':'remate (pendiente)'}

In [13]:
# Dataframe.
# id_equipo:equipo
dicc_equipos = dict(zip(laLiga.id_local, laLiga.local))

In [14]:
# Se añaden campos adicionales al dataframe
laLiga['equipo'] = laLiga['id_equipo']
laLiga['jugador'] = laLiga['id_jugador']
laLiga['categoria_evento'] = laLiga['id_categoria_evento']
laLiga['calificador'] = laLiga['id_calificador']
laLiga['jugador'] = laLiga['jugador'].map(dicc_jugadores)
laLiga['categoria_evento'] = laLiga['id_categoria_evento'].map(dicc_eventos)
laLiga['calificador'] = laLiga['id_calificador'].map(dicc_calificadores)
laLiga['equipo'] = laLiga['id_equipo'].map(dicc_equipos)

In [15]:
# Se reposicionan los campos del dataframe
laLiga = laLiga[['id_local','local','id_visitante','visitante','gol_local','gol_visitante','fecha','id_evento','id_periodo',
                 'minuto','segundo','id_categoria_evento','categoria_evento','id_calificador','calificador','valor','id_equipo',
                 'equipo','id_jugador','jugador','x','y','outcome']]

In [16]:
laLiga.shape

(2950881, 23)

In [29]:
cd C:\Users\DELUX\Desktop\publicaciones\3_GolesEsperados\dataframe

C:\Users\DELUX\Desktop\publicaciones\3_GolesEsperados\dataframe


In [30]:
# Se graba archivo final con dataframe completo con los eventos de LaLiga (temporada 2018-2019)
laLiga.to_csv('laLiga.csv',encoding='utf-8', index=False)